In [1]:
import pandas as pd
import os
import pathlib

In [3]:
datafile = '..\data\schedule.xlsx'
vac = pd.read_excel(datafile,sheet_name='Vacation')
days = pd.read_excel(datafile,sheet_name='Days')
weeks = list(set(days.week))
vac_sched = days.merge(vac,on='Date')

#build list of all potential months call work time units (week or weekend)
#and assign this an index
potential_call = days.callshiftassignment.drop_duplicates()\
    .reset_index().drop('index',axis=1).reset_index()


vac_slots = vac_sched[['week','callshiftassignment','Person']].drop_duplicates()\
    .sort_values(by=['week','Person'])[['callshiftassignment','Person']].drop_duplicates()

vac_slots = vac_slots.merge(potential_call,on='callshiftassignment')

In [4]:
#write the scheduled vacation weeks into a dict
#this shows which weeks/weekends people cannot work
vac_weeks = {}
for row in vac_slots.index:
    
    data = vac_slots.loc[row]
    if data[1] not in vac_weeks:
        vac_weeks[data[1]]= [data[2]]

    if data[1] in vac_weeks:
        if data[2] not in vac_weeks[data[1]]:
            vac_weeks[data[1]].append(data[2])

#determine which weeks/weekends people can work
#construct a dict of potential call weeks/weekends
not_vac_weeks = {}

for person in vac_weeks:
    for week in potential_call.index:

        #create first entry
        if person not in not_vac_weeks:
            if week not in vac_weeks[person]:
                not_vac_weeks[person] = [week]

        if person in not_vac_weeks:
            if week not in vac_weeks[person]:
                if week not in not_vac_weeks[person]:
                    not_vac_weeks[person].append(week)


In [5]:
residents = list(set(vac.Person))
shifts = list(potential_call.index)
weekends = list(potential_call[potential_call.callshiftassignment.str.contains('weekend')].index)
weekdays = [shift for shift in list(potential_call.index) if shift not in weekends]

In [6]:
import pulp

In [7]:
x = pulp.LpVariable.dicts(
    'x',
    ((weekday,weekend, resident)
        for resident in residents
        for weekday in weekdays
        for weekend in weekends),
    cat=pulp.LpBinary)

In [ ]:
model = pulp.LpProblem('Schedule', pulp.LpMaximize)

#constraints for residents
for resident in residents:

    # this is the "no time-turners constraint"
     
    #minumum number of shifts 
    model.addConstraint(
        sum(x[weekend, resident] for shift in shifts) >= 5,
        F'min {5} shifts per resident {resident}')

    model.addConstraint(
        sum(x[shift, resident] for shift in shifts) <= 7,
        F'max {7} shifts per resident {resident}')

    #remove vacation weeks
    for s in vac_weeks[resident]:
        model.addConstraint(
                sum(x[s, resident] for shift in shifts) == 0 ,
                '{} cannot work during {}'.format(resident,s))

    #no back to back call
    #no_two_shifts_in_a_row(resident)
    #remove vacation weeks
    for s in not_vac_weeks[resident]:
        for i in not_vac_weeks[resident][1:]:
            if i - s == 1:
                model.addConstraint(
                        sum(x[s, resident] for shift in shifts)
                        + sum(x[i, resident] for shift in shifts) <= len(shifts) ,
                        '{} cannot work during {}'.format(resident,s))

        for w in weekends:
            model.addConstraint(
                    sum(x[w, resident] for shift in shifts) <= 3 ,
                    '{} cannot work more than the {} weekends'.format(resident,w))    

            
    
        #no back to back call
    #no_two_shifts_in_a_row(resident)
    #remove vacation weeks
    #for s in not_vac_weeks[resident]:
    #    for i in weekends:
    #        if i == s:
    #            model.addConstraint(
    #                    sum(x[s, resident] for shift in shifts)
    #                    + sum(x[i, resident] for shift in shifts) <= len(shifts)/3 ,
    #                    '{} cannot work during {}'.format(resident,s))
    

    



for shift in shifts:
    
    #no duplication of shifts
    model.addConstraint(
        sum(x[shift, resident] for resident in residents) == 1,
        F'{1} resident per {shift}')



            #model.addConstraint(
            #        sum(x[minus, resident] for resident in residents)+ sum(x[s, resident] for resident in residents) == 0 ,
            #        '{} cannot work shift {} before {}'.format(resident,s,minus))
